En este archivo vamos a entrenar una red neuronal para predecir el resultado de los partidos de la Champions bsándonos en datos de los último 7 años.

Para empezar primero cargamos las librerias necesarias.

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from keras.layers import Dropout

Ahora vamos a empezar el entrenamiento para predecir resultados, pero antes de ello vamos a transformar las columnas string de equipos en numéricas para poder evaluarlas y analizarlas durante el procesamiento de la red neuronal.

In [39]:
# Cargar los datos del ranking UEFA
uefa_ranking = pd.read_csv("data/UEFA_Ranking.csv")

# Crear un diccionario para mapear nombres de equipos a números
team_mapping = {team: rank for rank, team in zip(uefa_ranking['Position'], uefa_ranking['Club'])}

# Cargar los datos de los partidos de la Champions League 2017
champions_data = pd.read_csv("data/champions-league-2017.csv")

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions_data['Home Team'] = champions_data['Home Team'].map(team_mapping)
champions_data['Away Team'] = champions_data['Away Team'].map(team_mapping)
champions_data['Winner'] = champions_data['Winner'].map(team_mapping)

Vamos a comprobar que todos los datos están bien.

In [40]:
# Ver las primeras filas de los datos
print(champions_data.head())

   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         20         61  1 - 2      61   
1             2            1         11         26  3 - 0      11   
2            13            1          2         52  3 - 0       2   
3            14            1         50          8  0 - 5       8   
4            25            1          9         66  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals  
0                1                2            3  
1                3                0            3  
2                3                0            3  
3                0                5            5  
4                6                0            6  


In [41]:
# analizar los datos
print(champions_data.describe())

       Match Number   Home Team   Away Team     Winner  Home Team Goals  \
count    125.000000  125.000000  125.000000  125.00000       125.000000   
mean      63.000000   25.288000   25.320000   17.24800         1.784000   
std       36.228442   28.268247   28.242784   22.06722         1.532361   
min        1.000000    1.000000    1.000000    1.00000         0.000000   
25%       32.000000    6.000000    6.000000    4.00000         1.000000   
50%       63.000000   14.000000   14.000000   10.00000         1.000000   
75%       94.000000   36.000000   36.000000   23.00000         3.000000   
max      125.000000  119.000000  119.000000  119.00000         7.000000   

       Away Team Goals  Match Goals  
count       125.000000   125.000000  
mean          1.424000     3.208000  
std           1.398525     1.622901  
min           0.000000     0.000000  
25%           0.000000     2.000000  
50%           1.000000     3.000000  
75%           2.000000     4.000000  
max           7.0000

In [42]:
# ver los datos nulos
print(champions_data.isnull().sum())

Match Number       0
Round Number       0
Home Team          0
Away Team          0
Result             0
Winner             0
Home Team Goals    0
Away Team Goals    0
Match Goals        0
dtype: int64


Como podemos ver está todo bien. Para más información con respecto al CSV que estamos tratando dirigirse a graficas.ipynb

Ahora vamos a comenzar el proceso de entrenamiento de la red neuronal

In [43]:
'''# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals']].values
y = champions_data['Winner'].values'''

# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals']].values
y = champions_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)

# Convertir etiquetas a one-hot encoding
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Definir modelo de red neuronal con más complejidad
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

# Compilar modelo con hiperparámetros ajustados
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar modelo
model.fit(X_train, y_train, epochs=70, batch_size=40, validation_split=0.1)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/70


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - accuracy: 0.0149 - loss: 3.3864 - val_accuracy: 0.2000 - val_loss: 3.3482
Epoch 2/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0535 - loss: 3.3635 - val_accuracy: 0.2000 - val_loss: 3.3500
Epoch 3/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0323 - loss: 3.3698 - val_accuracy: 0.0000e+00 - val_loss: 3.3526
Epoch 4/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0708 - loss: 3.3647 - val_accuracy: 0.0000e+00 - val_loss: 3.3532
Epoch 5/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0472 - loss: 3.3615 - val_accuracy: 0.0000e+00 - val_loss: 3.3532
Epoch 6/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1069 - loss: 3.3569 - val_accuracy: 0.0000e+00 - val_loss: 3.3514
Epoch 7/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0448 - loss: 3.3418 - val_accuracy: 0.0000e+00 - val_loss: 3.3482
Epoch 8/70
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0882 - loss: 3.3376 - val_accuracy: 0.0000e+00 - val

Vemos que el entrenamiento inicial ha ido sin problemas. Vamos a probar a ver que predicciones realizaría nuestra red neuronal en base a los partidos de la Champions utilizada para obtener una medida de precisión real. Para ello primero vamos a medir la precisión de una predicción en general.

In [44]:
# Hacer predicciones en los datos de prueba
predictions = model.predict(X_test)

# Convertir las predicciones de vuelta a etiquetas
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Calcular la precisión
accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy on test data:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Accuracy on test data: 0.08


Como puede verse con este resultado y la medición general anterior, la precisión del modelo es casi inexistente, por lo que vamos a seguir entrenándolo: